
# Spotify Party Queue — REST API Test Notebook

Dieses Notebook testet deine REST-API (`api.py`) mit folgenden Schritten:

1. **Queue abrufen** und ausgeben  
2. **Zwei Songs** zur Queue hinzufügen  
   - https://open.spotify.com/track/1c3T9VfcoJUblx0tnHf8Ha?si=f1325a6ae10e4146  
   - https://open.spotify.com/track/1OcV53oesLQw3VTW9I3uD3?si=d1c9e3677a4d425f  
3. **Queue abrufen** und ausgeben  
4. **Fünf zufällige Abstimmungen** (`+1` oder `-1`) auf einen der beiden Songs (zufällig gewählt) — inkl. `X-Client-Id`-Header  
5. **Queue abrufen** und ausgeben

> **Voraussetzung:** Dein FastAPI-Server läuft lokal, z. B.:  
> `uvicorn api:app --reload --port 8000`


In [9]:
from db_api import reset_db, _init_db
reset_db()
_init_db()

In [2]:

# Configuration
BASE_URL = "http://127.0.0.1:8000"

# Test track links (as specified)
TRACK_LINKS = [
    "https://open.spotify.com/track/5vNRhkKd0yEAg8suGBpjeY?si=b60c75d7407340da", # A
    "https://open.spotify.com/track/4kbkYbhWNiSJraySknB4hD?si=38b00ef76d9b4240", # B
    "https://open.spotify.com/track/6E0dc82Da4W5QWXR8ktoGT?si=644080da1b5c409b", # C
    "https://open.spotify.com/track/6ZjGVECiTZnZUnLqit0sUS?si=7e356a4808754191", # D
    "https://open.spotify.com/track/4AFbvNPzLlMPt36Q9fVgKZ?si=5c9dcdc4610e4440", # E
    "https://open.spotify.com/track/1v1cijv1qjMJ5o9OvMWACS?si=0fbe130021ce49f4", # F
    "https://open.spotify.com/track/1CPZ5BxNNd0n0nF4Orb9JS?si=a7cd57852d2b4295", # G
]

# Optional: fixed client id for reproducibility; set to None to randomize
FIXED_CLIENT_ID = None


In [11]:

import random
import uuid
import re
import requests
from typing import List, Dict, Any, Optional

def get_client_id() -> str:
    if FIXED_CLIENT_ID:
        return FIXED_CLIENT_ID
    return str(uuid.uuid4())

def pretty_print_queue(queue: List[Dict[str, Any]]) -> None:
    if not queue:
        print("Queue is empty []")
        return
    for i, item in enumerate(queue, 1):
        print(f"{i:02d}. {item.get('name')} — {item.get('artist')}  (id={item.get('id')}, votes={item.get("vote_sum")}, client_vote={item.get("client_vote")})")

SPOTIFY_TRACK_ID_RE = re.compile(r"""(?:
    spotify:track:(?P<id1>[A-Za-z0-9]+)
    |
    open\.spotify\.com/track/(?P<id2>[A-Za-z0-9]+)
)""", re.IGNORECASE | re.VERBOSE)

def extract_track_id(link_or_uri: str) -> Optional[str]:
    m = SPOTIFY_TRACK_ID_RE.search(link_or_uri or "")
    return m.group("id1") or m.group("id2") if m else None


## 1) Queue abrufen

In [4]:

resp = requests.get(f"{BASE_URL}/queue")
print("Status:", resp.status_code)
queue = resp.json() if resp.ok else []
pretty_print_queue(queue)


Status: 200
Queue is empty []


## 2) Zwei Songs zur Queue hinzufügen

In [5]:

for link in TRACK_LINKS:
    payload = {"song_link": link}
    resp = requests.post(f"{BASE_URL}/queue", json=payload)
    print(f"Add {link} → Status:", resp.status_code, "| Result:", resp.json() if resp.ok else resp.text)


Add https://open.spotify.com/track/5vNRhkKd0yEAg8suGBpjeY?si=b60c75d7407340da → Status: 200 | Result: True
Add https://open.spotify.com/track/4kbkYbhWNiSJraySknB4hD?si=38b00ef76d9b4240 → Status: 200 | Result: True
Add https://open.spotify.com/track/6E0dc82Da4W5QWXR8ktoGT?si=644080da1b5c409b → Status: 200 | Result: True
Add https://open.spotify.com/track/6ZjGVECiTZnZUnLqit0sUS?si=7e356a4808754191 → Status: 200 | Result: True
Add https://open.spotify.com/track/4AFbvNPzLlMPt36Q9fVgKZ?si=5c9dcdc4610e4440 → Status: 200 | Result: True
Add https://open.spotify.com/track/1v1cijv1qjMJ5o9OvMWACS?si=0fbe130021ce49f4 → Status: 200 | Result: True
Add https://open.spotify.com/track/1CPZ5BxNNd0n0nF4Orb9JS?si=a7cd57852d2b4295 → Status: 200 | Result: True


## 3) Queue erneut abrufen

In [6]:

resp = requests.get(f"{BASE_URL}/queue")
print("Status:", resp.status_code)
queue = resp.json() if resp.ok else []
pretty_print_queue(queue)


Status: 200
01. APT. — ROSÉ, Bruno Mars  (id=5vNRhkKd0yEAg8suGBpjeY, votes=0)
02. Baller — Abor & Tynna  (id=4kbkYbhWNiSJraySknB4hD, votes=0)
03. Call On Me - Radio Edit — Eric Prydz  (id=6E0dc82Da4W5QWXR8ktoGT, votes=0)
04. Deep Cries Out - Live — Christ For The Nations Worship, Gabriel Allred  (id=6ZjGVECiTZnZUnLqit0sUS, votes=0)
05. Ewigkeit — O'Bros, Outbreakband  (id=4AFbvNPzLlMPt36Q9fVgKZ, votes=0)
06. Fata Morgana — Nina Chuba  (id=1v1cijv1qjMJ5o9OvMWACS, votes=0)
07. Golden — HUNTR/X, EJAE, AUDREY NUNA, REI AMI, KPop Demon Hunters Cast  (id=1CPZ5BxNNd0n0nF4Orb9JS, votes=0)


## 4) Fünf zufällige Abstimmungen (+1 oder -1)

In [7]:

ids = []
for link in TRACK_LINKS:
    tid = extract_track_id(link)
    if tid:
        ids.append(tid)

if len(ids) < 2:
    print("Warnung: Konnte nicht beide Track-IDs extrahieren:", ids)

for i in range(100):
    client_id = get_client_id()
    print("Client-ID (X-Client-Id):", client_id)
    song_id = random.choice(ids)
    vote = random.choice([1, 0, -1])
    payload = {"song_id": song_id, "vote": vote}
    headers = {"X-Client-Id": client_id, "Content-Type": "application/json"}
    resp = requests.post(f"{BASE_URL}/vote", json=payload, headers=headers)
    print(f"Vote {i+1}/5 → song_id={song_id}, vote={vote} → Status:", resp.status_code, "| Result:", resp.json() if resp.ok else resp.text)


Client-ID (X-Client-Id): 93874988-3a99-4785-af58-f9d922a4429c
Vote 1/5 → song_id=6ZjGVECiTZnZUnLqit0sUS, vote=-1 → Status: 200 | Result: True
Client-ID (X-Client-Id): b6f7fb5a-951f-4c4a-91f9-08b4957bf83e
Vote 2/5 → song_id=4AFbvNPzLlMPt36Q9fVgKZ, vote=-1 → Status: 200 | Result: True
Client-ID (X-Client-Id): e1888b4c-3161-4633-91d5-c678cb1fadfe
Vote 3/5 → song_id=6ZjGVECiTZnZUnLqit0sUS, vote=-1 → Status: 200 | Result: True
Client-ID (X-Client-Id): ac0b871f-6f94-4469-a769-874052c0bc54
Vote 4/5 → song_id=6E0dc82Da4W5QWXR8ktoGT, vote=-1 → Status: 200 | Result: True
Client-ID (X-Client-Id): 33be35cb-855b-4393-980d-66d79f026c35
Vote 5/5 → song_id=4AFbvNPzLlMPt36Q9fVgKZ, vote=-1 → Status: 200 | Result: True
Client-ID (X-Client-Id): 8b60f138-e731-4061-8478-d294ef9994fd
Vote 6/5 → song_id=4kbkYbhWNiSJraySknB4hD, vote=0 → Status: 200 | Result: True
Client-ID (X-Client-Id): d1fa0ca0-2e0a-43e2-a989-780297759e6b
Vote 7/5 → song_id=1v1cijv1qjMJ5o9OvMWACS, vote=0 → Status: 200 | Result: True
Client-I

## 5) Queue abrufen (final)

In [34]:
client_id = get_client_id()
song_id = "5vNRhkKd0yEAg8suGBpjeY"
vote = random.choice([1])
payload = {"song_id": song_id, "vote": vote}
headers = {"X-Client-Id": client_id, "Content-Type": "application/json"}
resp = requests.post(f"{BASE_URL}/vote", json=payload, headers=headers)
headers = {"X-Client-Id": client_id, "Content-Type": "application/json"}
resp = requests.get(f"{BASE_URL}/queue", headers=headers)
print("Status:", resp.status_code)
queue = resp.json() if resp.ok else []
pretty_print_queue(queue)

Status: 200
01. APT. — ROSÉ, Bruno Mars  (id=5vNRhkKd0yEAg8suGBpjeY, votes=-1, client_vote=1)
